# BoltzDesign1 ⚡✨


For more details, read BoltzDesign1 paper **(https://www.biorxiv.org/content/10.1101/2025.04.06.647261v1)**

**❗ WARNING:** the following pipeline is in active development and has NOT been experimentally validated in lab. We are releasing the code to allow the community to contribute and build on.

**📧 Contact**
For feedback, questions or collaboration opportunities, please email yehlin@mit.edu

**➡️ Reference**
We implemented the visualization of designing trajectories using logMD (https://colab.research.google.com/drive/1-9GXUPna4T0VFlDz9I64gzRQz259_G8f?usp=sharing#scrollTo=4eXNO1JJHYrB)

In [1]:
#@title 🛠️ setup (~3 minutes)
%%time
import os, time, gc, io
import contextlib
from pathlib import Path

if not os.path.isdir("BoltzDesign1"):
  !git clone https://github.com/yehlincho/BoltzDesign1.git
  !cd /content/BoltzDesign1/boltz; pip install git+https://github.com/prody/ProDy.git .
  !pip install pypdb -qqq
  !pip install py3Dmol -qqq
  !pip install logmd -qqq
  !cd /content/BoltzDesign1/LigandMPNN && bash get_model_params.sh "./model_params"
  exit()

Cloning into 'BoltzDesign1'...
remote: Enumerating objects: 475, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 475 (delta 79), reused 67 (delta 64), pack-reused 377 (from 1)
Receiving objects: 100% (475/475), 492.06 KiB | 9.28 MiB/s, done.
Resolving deltas: 100% (232/232), done.
  Cloning https://github.com/prody/ProDy.git to /tmp/pip-req-build-gtr0kc94
  Running command git clone --filter=blob:none --quiet https://github.com/prody/ProDy.git /tmp/pip-req-build-gtr0kc94
  Resolved https://github.com/prody/ProDy.git to commit 4d75baaa769fd7a0173cfe18ea208bb6fb76bdcc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Processing /content/BoltzDesign1/boltz
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2

In [1]:
#@title 🛠️ load models
from boltz.main import download
from pathlib import Path
download(Path("."))

import os, sys
sys.path.append('./BoltzDesign1/boltzdesign')
from boltzdesign_utils import *
from ligandmpnn_utils import *

import pandas as pd
import os
import yaml
from pathlib import Path

ccd_path = "ccd.pkl"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
predict_args = {
    "recycling_steps": 0,  # Default value
    "sampling_steps": 200,  # Default value
    "diffusion_samples": 1,  # Default value
    "write_confidence_summary": True,
    "write_full_pae": False,
    "write_full_pde": False,
}

boltz_model = get_boltz_model('boltz1_conf.ckpt', predict_args, device)
boltz_model.train()
print("loaded model")

@> ProDy is configured: verbosity='none'
INFO:.prody:ProDy is configured: verbosity='none'


loaded model


In [2]:
#@title 🛠️ load utils
from input_utils import *
from utils import *

import requests
from pathlib import Path
from google.colab import files
from prody import parsePDB
import pypdb
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)


def generate_yaml_config(
    input_type,
    pdb_path,
    target_type,
    target_name,
    pdb_target_ids,
    target_mols,
    custom_target_input,
    custom_target_ids,
    binder_id,
    use_msa,
    contact_residues,
    modifications,
    modifications_positions,
    modification_target,
    constraint_target,
    config_obj
):
    """Generate YAML configuration based on input type"""
    # Process constraints if specified
    constraints = None
    modifications_dict = None
    if contact_residues or modifications:
        target_ids = pdb_target_ids if input_type == "pdb" else custom_target_ids
        target_ids_list = [str(x.strip()) for x in target_ids.split(",")] if target_ids else []
        target_id_map = {id: c for id, c in zip(target_ids_list, 'BCDEFGHIJKLMNOPQRSTUVWXYZ')}
        print(f"Mapped target IDs: {list(target_id_map.values())}")
        constraints, modifications_dict = process_design_constraints(
            target_id_map, modifications, modifications_positions,
            modification_target, contact_residues, constraint_target, binder_id
        )

    # Get target sequences
    target = []
    if input_type == "pdb":
        if pdb_path is not None:
            print("load local pdb from", pdb_path)
            if not Path(pdb_path).is_file():
                raise FileNotFoundError(f"Could not find local PDB: {pdb_path}")
        else:
            print("download pdb from RCSB")
            download_pdb(target_name, config_obj.PDB_DIR)
            pdb_path = config_obj.PDB_DIR / f"{target_name}.pdb"

        if target_type in ['rna', 'dna']:
            nucleotides = get_nucleotide_from_pdb(pdb_path)
            target = [nucleotides[id]['seq'] for id in pdb_target_ids.split(",")]
        elif target_type == 'small_molecule':
            ligands = get_ligand_from_pdb(target_name)
            target = [ligands[mol] for mol in target_mols.split(",")]
        elif target_type == 'protein':
            sequences = get_chains_sequence(pdb_path)
            target = [sequences[id] for id in pdb_target_ids.split(",")]
        else:
            raise ValueError(f"Unsupported target type: {target_type}")
    else:
        target = custom_target_input.split(",") if custom_target_input else [target_name]

    return generate_yaml_for_target_binder(
        target_name,
        target_type,
        target,
        config=config_obj,
        binder_id=binder_id,
        constraints=constraints,
        modifications=modifications_dict['data'] if modifications_dict else None,
        modification_target=modifications_dict['target'] if modifications_dict else None,
        use_msa=use_msa
    )

In [3]:
#@title 📄 Generate YAML file for protein design

%%time
import warnings, os, re
import yaml
warnings.simplefilter(action='ignore', category=FutureWarning)
# USER OPTIONS

#@markdown Select input type:
input_type = "pdb" #@param ["pdb", "custom"]
#@markdown Specify custom pdb path, otherwise will fetch from RCSB
pdb_path = "/content/BoltzDesign1/inputs/7v11.pdb" #@param {type:"string"}
#@markdown Select target type:
target_type = "small_molecule" #@param ["protein", "rna", "dna", "small_molecule", "metal"]
non_protein_target = False if target_type == "protein" else True
#@markdown Enter target name/PDB code:
target_name = "7v11" #@param {type:"string"}

#@markdown For PDB targets, protein and DNA/RNA enter target chain ID (optional), if multiple, comma-separated list(e.g. "A, B")
pdb_target_ids = "A" #@param {type:"string"}

#@markdown For PDB targets, small molecule enter target mol (optional), if multiple, comma-separated list(e.g. "SAM, FAD")
target_mols = "OQO" #@param {type:"string"}
#@markdown For custom input, enter target sequences/SMILES (comma-separated for multiple, (e.g. "ATGC, TACG")
custom_target_input = "" #@param {type:"string"}
#@markdown target ids (e.g. "B", "C")
custom_target_ids = "" #@param {type:"string"}

#@markdown Enter binder chain ID (default: A):
binder_id = "A" #@param {type:"string"}

#@markdown Use MSA; if False, runs in single-sequence mode
use_msa = False  #@param {type:"boolean"}
#@markdown Enter msa max seqs (default: 4096):
msa_max_seqs = 4096 #@param {type:"integer"}

#@markdown Add modifications? (comma-separated, e.g., "SEP, SEP")
modifications = "" #@param {type:"string"}
#@markdown Add modifications to which residues? (comma-separated, e.g., "S, S")
modifications_wt = "" #@param {type:"string"}
#@markdown Specify positions? (comma-separated, matching order, e.g., "2,3"
modifications_positions = "" #@param {type:"string"}
#@markdown Target ID for modifications (e.g., "B")
modification_target = "" #@param {type:"string"}

#@markdown Target ID for constraints (e.g., "B")
constraint_target = "" #@param {type:"string"}
#@markdown Specify positions? (Contact residues for constraints (comma-separated, e.g., "99,100,109")')
contact_residues = "" #@param {type:"string"}

pocket_conditioning =bool(contact_residues)

config = Config(main_dir=f'/content/inputs/test_data/{target_name}_binder')
config.setup_directories()

yaml_dict, yaml_dir = generate_yaml_config(
    input_type,
    pdb_path,
    target_type,
    target_name,
    pdb_target_ids,
    target_mols,
    custom_target_input,
    custom_target_ids,
    binder_id,
    use_msa,
    contact_residues,
    modifications,
    modifications_positions,
    modification_target,
    constraint_target,
    config
)

print("Generated YAML configuration:")
for key, value in yaml_dict.items():
    print(f"{key}:")
    if isinstance(value, dict):
        for k, v in value.items():
            if isinstance(v, list):
                print(f"  {k}:")
                for item in v:
                    print(f"    - {item}")
            else:
                print(f"  {k}: {v}")
    elif isinstance(value, list):
        for item in value:
            print(f"  - {item}")
    else:
        print(f"  {value}")
# Load initial config.yaml values

print("\n🎯 target_type:", target_type)
if target_type == 'small_molecule':
    config_path = "/content/BoltzDesign1/boltzdesign/configs/default_sm_config.yaml"
elif target_type == 'metal':
    config_path = "/content/BoltzDesign1/boltzdesign/configs/default_metal_config.yaml"
elif target_type in ('dna', 'rna'):
    config_path = "/content/BoltzDesign1/boltzdesign/configs/default_na_config.yaml"
elif target_type == 'protein':
    config_path = "/content/BoltzDesign1/boltzdesign/configs/default_ppi_config.yaml"

print("📂 Using config path:", config_path)
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

main_dir = 'outputs'
os.makedirs(main_dir, exist_ok=True)
version_name = f'{target_type}_{target_name}'
print(f"\n💾 YAML config saved at: {yaml_dir}")
print(f"🎨 Design outputs will be saved in: {main_dir}")

DEBUG:.prody:Connecting wwPDB FTP server RCSB PDB (USA).
INFO:.prody:Downloading PDB files via FTP failed, trying HTTP.


load local pdb from /content/BoltzDesign1/inputs/7v11.pdb


DEBUG:.prody:7v11 downloaded (7v11.pdb.gz)
DEBUG:.prody:PDB download via HTTP completed (1 downloaded, 0 failed).
DEBUG:.prody:2139 atoms and 1 coordinate set(s) were parsed in 0.03s.


Generated YAML configuration:
version:
  1
sequences:
  - {'protein': {'id': ['A'], 'sequence': 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX', 'msa': 'empty'}}
  - {'ligand': {'id': ['B'], 'smiles': 'Fc1c(Cl)ccc(n2cnnn2)c1c1c[n+]([O-])c(cc1)C(CC1CC1)n1cc(cn1)c1ccc(N)nc1C'}}

🎯 target_type: small_molecule
📂 Using config path: /content/BoltzDesign1/boltzdesign/configs/default_sm_config.yaml

💾 YAML config saved at: /content/inputs/test_data/7v11_binder/yaml/7v11.yaml
🎨 Design outputs will be saved in: outputs
CPU times: user 74.4 ms, sys: 6.12 ms, total: 80.5 ms
Wall time: 1.85 s


## 2. Run BoltzDesign protocol

In [4]:
#@title 🧬 Protein design settings

# Always update other shared parameters
#@markdown Number of designs to generate
design_samples = 1  #@param {type:"integer"}
#@markdown Select minimum and maximum length (default min : 100, max: 150)
length_min = 250  #@param {type:"integer"}
length_max = 300  #@param {type:"integer"}

#@markdown Select optimizer (default: SGD)
optimizer_type = "SGD"  #@param ["SGD", "AdamW"]

config['binder_chain'] = binder_id
config['non_protein_target'] = non_protein_target
config['msa_max_seqs'] = msa_max_seqs
config['length_min'] = length_min
config['length_max'] = length_max
config['optimizer_type'] = optimizer_type
config['pocket_conditioning'] = pocket_conditioning

In [5]:
#@title ⚙️ (Optional: Change protein design settings)

# @markdown 🌟 **Use default config?** (if False, override with param you change)
use_default_config = True  # @param {type:"boolean"}
# @markdown ---
print("\n🔧 **Protein Design Settings**")

# If user chooses to override, they can set these params below:
# @markdown Mask target for warm-up stage
mask_ligand = True  # @param {type:"boolean"}

# @markdown Optimize interface contact per binder position
# (default: False for small molecule and metal, True for protein and NA)
optimize_contact_per_binder_pos = False  # @param {type:"boolean"}

# @markdown Only use distogram for optimization (else use both pairformer and confidence modules)
distogram_only = True  # @param {type:"boolean"}

# @markdown Design algorithm
design_algorithm = "3stages"  # @param ["3stages", "3stages_extra"]

# @markdown Softmax temperature for 3stages (logits to softmax, T = e_soft)
e_soft = 0.8  # @param {type:"number"}

# @markdown Softmax temperature for 3stages_extra (Initial logits to softmax, T = e_soft_1. Additional logits to softmax, T = e_soft_2)
e_soft_1 = 0.8  # @param {type:"number"}
e_soft_2 = 1.0  # @param {type:"number"}

# @markdown Learning rate for pre-stage (warm-up)
learning_rate_pre = 0.1  # @param {type:"number"}

# @markdown Learning rate for soft to hard stages
learning_rate = 0.1  # @param {type:"number"}

# @markdown Select interaction distance cutoff and number of inter/intra contacts
# @markdown e.g. for metal, change `num_inter_contacts` based on coordination number

inter_chain_cutoff = 20  # @param {type:"integer"}
intra_chain_cutoff = 14  # @param {type:"integer"}
num_inter_contacts = 2  # @param {type:"integer"}
num_intra_contacts = 2  # @param {type:"integer"}

# @markdown Select helix loss
helix_loss_max = 0.0  # @param {type:"number"}
helix_loss_min = -0.3  # @param {type:"number"}

# === Default values ===
default_params = {
    'mask_ligand': True,
    'optimize_contact_per_binder_pos': False,
    'distogram_only': True,
    'design_algorithm': "3stages",
    'e_soft': 0.8,
    'e_soft_1': 0.8,
    'e_soft_2': 1.0,
    'inter_chain_cutoff': 20,
    'intra_chain_cutoff': 14,
    'num_inter_contacts': 2,
    'num_intra_contacts': 2,
    'learning_rate_pre': 0.1,
    'learning_rate': 0.1,
    'helix_loss_max': 0.0,
    'helix_loss_min': -0.3
}

if not use_default_config:
    # Collect current user-specified parameters
    current_params = {
        'mask_ligand': mask_ligand,
        'optimize_contact_per_binder_pos': optimize_contact_per_binder_pos,
        'distogram_only': distogram_only,
        'design_algorithm': design_algorithm,
        'e_soft': e_soft,
        'e_soft_1': e_soft_1,
        'e_soft_2': e_soft_2,
        'inter_chain_cutoff': inter_chain_cutoff,
        'intra_chain_cutoff': intra_chain_cutoff,
        'num_inter_contacts': num_inter_contacts,
        'num_intra_contacts': num_intra_contacts,
        'learning_rate_pre': learning_rate_pre,
        'learning_rate': learning_rate,
        'helix_loss_max': helix_loss_max,
        'helix_loss_min': helix_loss_min
    }

    # Update only changed parameters
    updated_keys = []
    for key, val in current_params.items():
        if val != default_params[key]:
            config[key] = val
            updated_keys.append(key)

    if updated_keys:
        print(f"✅ Updated config parameters: {', '.join(updated_keys)}")
    else:
        print("⚠️ No config parameters updated; all match defaults.")
else:
    print("ℹ️ Using default config; no updates applied.")


🔧 **Protein Design Settings**
ℹ️ Using default config; no updates applied.


In [6]:
#@title 📋 Show Config

items = list(config.items())
max_key_len = max(len(key) for key, _ in items)
max_val_len = max(len(str(val)) for _, val in items)

# Print header
print("  🔧 " + "=" * (max_key_len + max_val_len + 5))
print("  📦 Current Config Settings")
print("  🔧 " + "=" * (max_key_len + max_val_len + 5))

# Print items in two columns
for i in range(0, len(items), 2):
    key1, value1 = items[i]
    if i + 1 < len(items):
        key2, value2 = items[i + 1]
        print(f"  🏷️ {key1:<{max_key_len}}: {str(value1):<{max_val_len}}    "
              f"🏷️ {key2:<{max_key_len}}: {value2}")
    else:
        print(f"  🏷️ {key1:<{max_key_len}}: {value1}")

print("  🔧 " + "=" * (max_key_len + max_val_len + 5))


  🔧 ===========================================
  📦 Current Config Settings
  🔧 ===========================================
  🏷️ binder_chain                   : A          🏷️ design_algorithm               : 3stages
  🏷️ disconnect_feats               : True       🏷️ disconnect_pairformer          : False
  🏷️ distogram_only                 : True       🏷️ e_soft                         : 0.8
  🏷️ e_soft_1                       : 0.8        🏷️ e_soft_2                       : 1.0
  🏷️ hard_iteration                 : 5          🏷️ helix_loss_max                 : -0.0
  🏷️ helix_loss_min                 : -0.3       🏷️ increasing_contact_over_itr    : False
  🏷️ inter_chain_cutoff             : 20.0       🏷️ intra_chain_cutoff             : 14.0
  🏷️ learning_rate                  : 0.1        🏷️ learning_rate_pre              : 0.2
  🏷️ length_max                     : 300        🏷️ length_min                     : 250
  🏷️ mask_ligand                    : True       🏷️ msa_max_seqs 

In [7]:
#@title 🔄 Design Iterations Setting
# @markdown ---
# @markdown Number of pre-iterations (warm-up stage)
pre_iteration = 30  # @param {type:"integer"}

# @markdown Number of Logits → Softmax iterations
soft_iteration = 75  # @param {type:"integer"}

# @markdown Number of Softmax (high T) → Softmax (low T) iterations
temp_iteration = 45  # @param {type:"integer"}

# @markdown Number of one-hot encoded iterations
hard_iteration = 5  # @param {type:"integer"}

# @markdown Number of MCMC mutation (semi-greedy) steps
semi_greedy_steps = 2  # @param {type:"integer"}

# @markdown ---
# @markdown Optional (for 3stages_extra algorithm only)
soft_iteration_1 = 50  # @param {type:"integer"}
soft_iteration_2 = 50  # @param {type:"integer"}

# Apply settings based on the chosen design algorithm
if design_algorithm == '3stages':
    config['soft_iteration'] = soft_iteration
elif design_algorithm == '3stages_extra':
    config['soft_iteration_1'] = soft_iteration_1
    config['soft_iteration_2'] = soft_iteration_2

config['pre_iteration'] = pre_iteration
config['temp_iteration'] = temp_iteration
config['hard_iteration'] = hard_iteration
config['semi_greedy_steps'] = semi_greedy_steps


In [8]:
#@title 🔥 Design Loss Setting
# @markdown ---
# @markdown Contact loss within protein (binder) chain
con_loss = 1  # @param {type:"number"}

# @markdown Inter contact loss between protein and target
i_con_loss = 1  # @param {type:"number"}

plddt_loss = 0.1  # @param {type:"number"}

pae_loss = 0.4  # @param {type:"number"}

i_pae_loss = 0.1  # @param {type:"number"}

rg_loss = 0.0  # @param {type:"number"}

loss_scales = {
    'con_loss': con_loss,
    'i_con_loss': i_con_loss,
    'plddt_loss': plddt_loss,
    'pae_loss': pae_loss,
    'i_pae_loss': i_pae_loss,
    'rg_loss': rg_loss,
}


In [ ]:
#@title Ready to Run! 🚀
import warnings
warnings.simplefilter("ignore", DeprecationWarning)

num_workers = 0  #@param {type:"integer"}
show_animation = True  #@param {type:"boolean"}
# @markdown Saves the design trajectory at every iteration. ⚠ **Note:** If you run with `distogram_only` mode, it does not generate coordinates — so enabling `save_trajectory` forces the model to predict coordinates every iteration, which requires more time.
save_trajectory = False  #@param {type:"boolean"}
# @markdown Keep this as False in Colab
redo_boltz_predict = False  #@param {type:"boolean"}
boltz_path = shutil.which("boltz")

run_boltz_design(
    boltz_path=boltz_path,
    main_dir=main_dir,
    yaml_dir=os.path.dirname(yaml_dir),
    boltz_model=boltz_model,
    ccd_path=ccd_path,
    design_samples=design_samples,
    version_name=version_name,
    config=config,
    loss_scales=loss_scales,
    num_workers=num_workers,  # Number of worker processes
    show_animation=show_animation,      # Show visual animation
    save_trajectory=save_trajectory,     # Save full trajectory + coordinates
    redo_boltz_predict=redo_boltz_predict,  # Given final designs, repredict with the boltz predict command
)

pre-run warm up
set in train mode


/content/./BoltzDesign1/boltzdesign/boltzdesign_utils.py:597: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return grad * torch.sqrt(torch.tensor(eff_L)) / (gn + 1e-7)


Epoch 0: lr: 0.200, soft: 1.00, hard: 0.00, temp: 1.00, total loss: 4.93, ['con_loss:5.17', 'helix_loss:0.84']
Epoch 1: lr: 0.200, soft: 1.00, hard: 0.00, temp: 1.00, total loss: 4.56, ['con_loss:4.91', 'helix_loss:1.22']
Epoch 2: lr: 0.200, soft: 1.00, hard: 0.00, temp: 1.00, total loss: 4.78, ['con_loss:5.06', 'helix_loss:0.97']
Epoch 3: lr: 0.200, soft: 1.00, hard: 0.00, temp: 1.00, total loss: 4.63, ['con_loss:4.94', 'helix_loss:1.08']
Epoch 4: lr: 0.200, soft: 1.00, hard: 0.00, temp: 1.00, total loss: 4.59, ['con_loss:5.07', 'helix_loss:1.72']
Epoch 5: lr: 0.200, soft: 1.00, hard: 0.00, temp: 1.00, total loss: 4.63, ['con_loss:4.94', 'helix_loss:1.10']
Epoch 6: lr: 0.200, soft: 1.00, hard: 0.00, temp: 1.00, total loss: 4.60, ['con_loss:5.13', 'helix_loss:1.88']
Epoch 7: lr: 0.200, soft: 1.00, hard: 0.00, temp: 1.00, total loss: 4.64, ['con_loss:4.86', 'helix_loss:0.80']
Epoch 8: lr: 0.200, soft: 1.00, hard: 0.00, temp: 1.00, total loss: 4.72, ['con_loss:4.95', 'helix_loss:0.82']
E

In [1]:
#@title 📊 Design Results
df = pd.read_csv(os.path.join(main_dir, version_name, 'results_final', 'rmsd_results.csv'))
df

NameError: name 'pd' is not defined

##3. Run LigandMPNN for redesign

In [ ]:
#@title Update LigandMPNN config file
yaml_path = "/content/BoltzDesign1/LigandMPNN/run_ligandmpnn_logits_config.yaml"
cwd = "/content/BoltzDesign1"
with open(yaml_path, "r") as f:
    mpnn_config = yaml.safe_load(f)
for key, value in mpnn_config.items():
    if isinstance(value, str) and "${CWD}" in value:
        mpnn_config[key] = value.replace("${CWD}", cwd)
assert Path(mpnn_config["checkpoint_soluble_mpnn"]).exists(), "Checkpoint file not found!"
with open(yaml_path, "w") as f:
    yaml.dump(mpnn_config, f, default_flow_style=False)
print("✅ Config updated and saved.")

✅ Config updated and saved.


In [ ]:
#@title Set the interface cutoff for redesign and run LigandMPNN
#@markdown For LigandMPNN redesign, specify number of designs per PDB
num_designs = 2 #@param {type:"integer"}
#@markdown Cutoff distance in Angstroms between target and binder atoms to define interface residues (default = 4 Å). If set into 0Å, fully redesign the sequence with LigandMPNN
cutoff = 4 #@param {type:"integer"}
#@markdown Cutoff iPTM for selecting proteins for redesign from the initial BoltzDesign.
i_ptm_cutoff = 0.5 #@param {type:"number"}
#@markdown For protein targets, enter target chain IDs (optional). If not specified, all chains except binder will be considered targets
target_ids = "" #@param  {type:"string"}
target_ids = [str(x.strip()) for x in target_ids.split(",")] if target_ids else None
boltzdesign_dir = main_dir + '/'+   version_name+'/results_final'
pdb_save_dir = main_dir +   '/'+ version_name+'/pdb'
ligandmpnn_dir = main_dir + '/'+ version_name+'/ligandmpnn_cutoff'
ligandmpnn_config= '/content/BoltzDesign1/LigandMPNN/run_ligandmpnn_logits_config.yaml'
os.makedirs(ligandmpnn_dir, exist_ok=True)
convert_cif_files_to_pdb(boltzdesign_dir, pdb_save_dir, high_iptm = True, i_ptm_cutoff=i_ptm_cutoff)
run_ligandmpnn_redesign(ligandmpnn_dir, pdb_save_dir, boltz_path,
    os.path.dirname(yaml_dir), ligandmpnn_config, top_k=num_designs, cutoff=cutoff, non_protein_target=non_protein_target, binder_chain=binder_id, target_chains='all')

In [ ]:
#@title 🏆🎉Download highly confident designs

import os
import json
import shutil
from zipfile import ZipFile
from google.colab import files

# --- Parameters ---
i_ptm_cutoff = 0.5 #@param {type:"number"}
complex_plddt_cutoff = 0.7 #@param {type:"number"}

# --- Directories ---
ligandmpnn_dir_boltz = os.path.join(ligandmpnn_dir, '01_lmpnn_redesigned')
yaml_dir_boltz = os.path.join(ligandmpnn_dir, '01_lmpnn_redesigned_yaml')
high_iptm_designs_dir = os.path.join(ligandmpnn_dir, '01_lmpnn_redesigned_high_iptm')

high_iptm_designs_dir_yaml = os.path.join(high_iptm_designs_dir, 'yaml')
high_iptm_designs_dir_cif = os.path.join(high_iptm_designs_dir, 'cif')

# --- Create output directories ---
os.makedirs(high_iptm_designs_dir_yaml, exist_ok=True)
os.makedirs(high_iptm_designs_dir_cif, exist_ok=True)

copied_any = False

# --- Process designs ---
for root in os.listdir(ligandmpnn_dir_boltz):
    root_path = os.path.join(ligandmpnn_dir_boltz, root, 'predictions')
    if not os.path.isdir(root_path):
        continue

    for subdir in os.listdir(root_path):
        json_path = os.path.join(root_path, subdir, f'confidence_{subdir}_model_0.json')
        yaml_path = os.path.join(yaml_dir_boltz, f'{subdir}.yaml')
        cif_path = os.path.join(ligandmpnn_dir_boltz, f'boltz_results_{subdir}', 'predictions', subdir, f'{subdir}_model_0.cif')

        try:
            with open(json_path, 'r') as f:
                data = json.load(f)

            design_name = json_path.split('/')[-2]
            length = int(subdir[subdir.find('length') + 6:subdir.find('_model')])
            iptm = data.get('iptm', 0)
            complex_plddt = data.get('complex_plddt', 0)

            print(f"{design_name} length: {length} complex_plddt: {complex_plddt:.2f} iptm: {iptm:.2f}")

            if iptm > i_ptm_cutoff and complex_plddt > complex_plddt_cutoff:
                shutil.copy(yaml_path, os.path.join(high_iptm_designs_dir_yaml, f'{subdir}.yaml'))
                shutil.copy(cif_path, os.path.join(high_iptm_designs_dir_cif, f'{subdir}.cif'))
                print(f"✅ {design_name} copied")
                copied_any = True

        except (KeyError, FileNotFoundError, json.JSONDecodeError) as e:
            print(f"⚠️ Skipping {subdir}: {e}")
            continue

# --- Zip and download if any files were copied ---
if copied_any:
    zip_filename = os.path.join(ligandmpnn_dir, 'high_confidence_designs.zip')
    with ZipFile(zip_filename, 'w') as zipf:
        for fname in os.listdir(high_iptm_designs_dir_yaml):
            fpath = os.path.join(high_iptm_designs_dir_yaml, fname)
            zipf.write(fpath, arcname=os.path.join('yaml', fname))
        for fname in os.listdir(high_iptm_designs_dir_cif):
            fpath = os.path.join(high_iptm_designs_dir_cif, fname)
            zipf.write(fpath, arcname=os.path.join('cif', fname))

    print(f"📦 Zipped results saved to {zip_filename}")
    files.download(zip_filename)
else:
    print("⚠️ No high-confidence designs found. Skipping zip and download.")


## ✅ Work in progress (currently implementing in a pipeline)

In [ ]:
## PyRosetta for protein–protein interface energy/interaction evaluation
print("Installing PyRosetta")
os.system("pip install pyrosettacolabsetup")
with contextlib.redirect_stdout(io.StringIO()):
  import pyrosettacolabsetup
  pyrosettacolabsetup.install_pyrosetta(serialization=True, cache_wheel_on_google_drive=False)

## Gnina for small-molecule docking score
!git clone https://github.com/dkoes/openbabel.git
!cd openbabel && mkdir build && cd build && cmake -DWITH_MAEPARSER=OFF -DWITH_COORDGEN=OFF -DPYTHON_BINDINGS=ON -DRUN_SWIG=ON ..&& make && make install
!git clone https://github.com/gnina/gnina.git
!cd gnina && mkdir build && cd build && cmake .. && make && make install